In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Define the directory containing your CSV files
data_dir = '/kaggle/input/sec-cameco/' if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ else '../data/tables_1009001/'

for dirname, _, filenames in os.walk(data_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

../data/tables_1009001/dim.csv
../data/tables_1009001/tag.csv
../data/tables_1009001/pre.csv
../data/tables_1009001/num.csv
../data/tables_1009001/ren.csv
../data/tables_1009001/sub.csv
../data/tables_1009001/cal.csv
../data/tables_1009001/txt.csv


In [2]:
import duckdb
import os

sec_pk = {'sub': 'adsh', 'tag': 'tag, version', 'ren': 'adsh, report', 'pre': 'adsh, report, line', 'cal': 'adsh, grp, arc', 'dim': 'dimhash'}
sec_date_column = {'sub': ['filed', 'floatdate']}

# Create an in-memory DuckDB connection
# con = duckdb.connect(database=':memory:', read_only=False)
con = duckdb.connect(database='../kaggle.db')

# List all files in the specified directory
all_files = os.listdir(data_dir)

# Filter for CSV files (you might need to adjust the extension if your files are different)
csv_files = [f for f in all_files if f.endswith('.csv')]

# Iterate through the CSV files and load them into DuckDB tables
for file_name in csv_files:
    file_path = os.path.join(data_dir, file_name)
    table_name = os.path.splitext(file_name)[0]  # Use the filename (without extension) as the table name

    try:
        # Option 1: Use pandas to read the CSV file into a DataFrame
        #df = pd.read_csv(file_path)

        # Load the DataFrame into a DuckDB table
        #con.register(table_name, df)  # Register the DataFrame as a view

        # Option 2: Using COPY FROM (Faster for large files)
        con.execute(f"""CREATE TABLE IF NOT EXISTS {table_name} AS SELECT * FROM read_csv('{file_path}', AUTO_DETECT=TRUE);""")
        if table_name in sec_date_column:
            strptimes: str = ""
            for col in sec_date_column[table_name]:
                strptimes += f"STRPTIME(CAST({col} AS VARCHAR), '%Y%m%d')::DATE AS {col}, \n"
            sql_command = f"""SELECT *, {strptimes.rstrip(', \n')} FROM {table_name};"""
            print(sql_command)
            con.execute(sql_command)
        if table_name in sec_pk:
            # Add primary key constraint
            sql_pk = f"ALTER TABLE {table_name} ADD PRIMARY KEY ({sec_pk[table_name]});"
            print(sql_pk)
            con.execute(sql_pk)


        print(f"Loaded '{file_name}' into DuckDB view '{table_name}'")

    except Exception as e:
        print(f"Error loading '{file_name}': {e}")

# Now you can query the data in DuckDB using SQL
# For example, to select the first few rows of one of your tables:
# table_to_query = csv_files[0].split('.')[0] # Get the name of the first table
# result = con.execute(f"SELECT * FROM {table_to_query} LIMIT 5;").fetchdf()
# print(result)

ALTER TABLE dim ADD PRIMARY KEY (dimhash);
Loaded 'dim.csv' into DuckDB view 'dim'
ALTER TABLE tag ADD PRIMARY KEY (tag, version);
Loaded 'tag.csv' into DuckDB view 'tag'
ALTER TABLE pre ADD PRIMARY KEY (adsh, report, line);
Loaded 'pre.csv' into DuckDB view 'pre'
Loaded 'num.csv' into DuckDB view 'num'
ALTER TABLE ren ADD PRIMARY KEY (adsh, report);
Loaded 'ren.csv' into DuckDB view 'ren'
SELECT *, STRPTIME(CAST(filed AS VARCHAR), '%Y%m%d')::DATE AS filed, 
STRPTIME(CAST(floatdate AS VARCHAR), '%Y%m%d')::DATE AS floatdate FROM sub;
ALTER TABLE sub ADD PRIMARY KEY (adsh);
Loaded 'sub.csv' into DuckDB view 'sub'
ALTER TABLE cal ADD PRIMARY KEY (adsh, grp, arc);
Loaded 'cal.csv' into DuckDB view 'cal'
Loaded 'txt.csv' into DuckDB view 'txt'


In [4]:

def print_duckdb_schema(conn: duckdb.DuckDBPyConnection, table_name: str):
  """Prints the schema of a specified table in a DuckDB database.

  Args:
    conn: An active DuckDB connection object.
    table_name: The name of the table whose schema to print.
  """
  try:
    schema_info = conn.execute(f"PRAGMA table_info('{table_name}')").fetchall()
    if not schema_info:
      print(f"Table '{table_name}' not found.")
      return

    print(f"Schema for table: {table_name}")
    print("-" * (16 + len(table_name)))
    print(f"{'Column ID':<10} {'Name':<20} {'Type':<15} {'NotNull':<8} {'PrimaryKey':<12} {'Default':<20}")
    print("-" * 85)
    for column in schema_info:
      cid, name, dtype, notnull, pk, default = column
      print(f"{cid:<10} {name:<20} {dtype:<15} {bool(notnull):<8} {bool(pk):<12} {str(default):<20}")
    print("-" * 85)

  except duckdb.CatalogException as e:
    print(f"Error: {e}")
    print(f"Could not retrieve schema for table '{table_name}'.")



In [5]:
result = con.execute(f"SELECT * FROM sub;")
print(result.fetchdf())

print_duckdb_schema(con, 'sub')

                   adsh      cik         name     sic countryba stprba  \
0  0001193125-24-075464  1009001  CAMECO CORP  1090.0        CA     SK   
1  0001193125-20-088317  1009001  CAMECO CORP  1090.0        CA     SK   
2  0001193125-21-087174  1009001  CAMECO CORP  1090.0        CA     SK   
3  0001193125-22-081448  1009001  CAMECO CORP  1090.0        CA     SK   
4  0001193125-23-083565  1009001  CAMECO CORP  1090.0        CA     SK   
5  0001193125-18-099445  1009001  CAMECO CORP  1090.0        CA     SK   
6  0001193125-19-092356  1009001  CAMECO CORP  1090.0        CA     SK   

      cityba    zipba            bas1  bas2  ...            accepted prevrpt  \
0  SASKATOON  S7M 1J3  2121 11TH ST W  None  ... 2024-03-22 17:23:00       0   
1  SASKATOON  S7M 1J3  2121 11TH ST W  None  ... 2020-03-27 14:02:00       0   
2  SASKATOON  S7M 1J3  2121 11TH ST W  None  ... 2021-03-19 13:08:00       0   
3  SASKATOON  S7M 1J3  2121 11TH ST W  None  ... 2022-03-22 14:05:00       0   
4  SASK

In [ ]:
# Remember to close the DuckDB connection when you're finished
con.close()